In [104]:
# Here I am using gensim for learning the word embeddings from the EUADR corpus
# This is just a rough start to learn 
import gensim
import logging
import os
import pandas as pd

In [105]:
# start the log
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO)

In [106]:
data_path

'/home/santosh/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/'

In [107]:
# get the current working directory
data_path = os.path.abspath(os.path.join(os.path.dirname( '__file__' ), '..', 'Datasets'))+'/'

# Although the dataset says csv, it is tab delimited. In addition to this, they have severe codels problems. 
# So best to parse throught codes first. 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 2: invalid start byte

#open for reading with "universal" type set

import codecs

doc_d_t = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_target'+'.csv','rU','UTF-8') 
EUADR_drug_target = pd.read_csv(doc_d_t, sep='\t')


doc_t_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_target_disease'+'.csv','rU','UTF-8',errors='ignore') 
EUADR_target_disease = pd.read_csv(doc_t_d, sep='\t')
       
doc_d_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_disease'+'.csv','rU','UTF-8')                       
EUADR_drug_disease = pd.read_csv(doc_d_d, sep='\t')

logging.info("Done reading data files")

2019-06-19 15:57:56,489 : INFO : Done reading data files


In [108]:
EUADR_target_disease.columns

Index(['ASSOCIATION_TYPE', 'PMID', 'NUM_SENTENCE', 'ENTITY1_TEXT',
       'ENTITY1_INI', 'ENTITY1_END', 'ENTITY1_TYPE', 'ENTITY2_TEXT',
       'ENTITY2_INI', 'ENTITY2_END', 'ENTITY2_TYPE', 'SENTENCE'],
      dtype='object')

In [109]:
sentences =  EUADR_drug_target['SENTENCE'].tolist() + EUADR_target_disease['SENTENCE'].tolist()+ EUADR_drug_disease['SENTENCE'].tolist()


In [113]:
sentences

['Gefitinib inhibition of drug resistance to doxorubicin by inactivating ABCG2 in thyroid cancer cell lines.',
 'Inactivation of EGFR kinase by gefitinib was analyzed by Western blot analysis and immunofluorescence.',
 'Gefitinib inhibition of drug resistance to doxorubicin by inactivating ABCG2 in thyroid cancer cell lines.',
 'Extrusion assays using flow cytometry analysis were used to determine the ability of thyroid cancer cells to extrude the chemotherapy drug, doxorubicin, via the ABCG2 drug transporter in the presence or absence of gefitinib.',
 'Inactivation of the EGFR kinase by gefitinib potentiates the cytotoxic effect of doxorubicin in thyroid cancer, most likely by decreasing the ability of the cell to extrude doxorubicin.',
 'Inhibition of EGFR kinase activity by gefitinib causes the translocation of the ABCG2 drug transporter away from the plasma membrane, resulting in a concomitant decrease in doxorubicin extrusion in thyroid cancer cell lines.',
 'A terminal deoxynucle

In [114]:
#########################  tokenisation ###############################################
documents = []
for row in sentences:
    try:
        documents.append(row.lower().split())
    except AttributeError:
        pass


In [116]:
documents

[['gefitinib',
  'inhibition',
  'of',
  'drug',
  'resistance',
  'to',
  'doxorubicin',
  'by',
  'inactivating',
  'abcg2',
  'in',
  'thyroid',
  'cancer',
  'cell',
  'lines.'],
 ['inactivation',
  'of',
  'egfr',
  'kinase',
  'by',
  'gefitinib',
  'was',
  'analyzed',
  'by',
  'western',
  'blot',
  'analysis',
  'and',
  'immunofluorescence.'],
 ['gefitinib',
  'inhibition',
  'of',
  'drug',
  'resistance',
  'to',
  'doxorubicin',
  'by',
  'inactivating',
  'abcg2',
  'in',
  'thyroid',
  'cancer',
  'cell',
  'lines.'],
 ['extrusion',
  'assays',
  'using',
  'flow',
  'cytometry',
  'analysis',
  'were',
  'used',
  'to',
  'determine',
  'the',
  'ability',
  'of',
  'thyroid',
  'cancer',
  'cells',
  'to',
  'extrude',
  'the',
  'chemotherapy',
  'drug,',
  'doxorubicin,',
  'via',
  'the',
  'abcg2',
  'drug',
  'transporter',
  'in',
  'the',
  'presence',
  'or',
  'absence',
  'of',
  'gefitinib.'],
 ['inactivation',
  'of',
  'the',
  'egfr',
  'kinase',
  'by',

In [144]:
model = gensim.models.Word2Vec(
        documents,
        size=50,
        window=30,
        min_count=0,
        workers=10)

model.train(documents, total_examples=len(documents), epochs=100)

2019-06-19 16:24:52,509 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-06-19 16:24:52,513 : INFO : collecting all words and their counts
2019-06-19 16:24:52,515 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-19 16:24:52,527 : INFO : collected 3704 word types from a corpus of 23456 raw words and 846 sentences
2019-06-19 16:24:52,530 : INFO : Loading a fresh vocabulary
2019-06-19 16:24:52,544 : INFO : effective_min_count=0 retains 3704 unique words (100% of original 3704, drops 0)
2019-06-19 16:24:52,547 : INFO : effective_min_count=0 leaves 23456 word corpus (100% of original 23456, drops 0)
2019-06-19 16:24:52,563 : INFO : deleting the raw counts dictionary of 3704 items
2019-06-19 16:24:52,566 : INFO : sample=0.001 downsamples 35 most-common words
2019-06-19 16:24:52,568 : INFO : downsampling leaves estimated 19182 word corpus (81.8% of prior 23456)
2019-06-19 16:24:52,577 : INFO : estimated required memory f

2019-06-19 16:24:53,010 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-06-19 16:24:53,012 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-06-19 16:24:53,014 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-06-19 16:24:53,017 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-06-19 16:24:53,019 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-06-19 16:24:53,020 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-06-19 16:24:53,022 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-06-19 16:24:53,024 : INFO : EPOCH - 2 : training on 23456 raw words (19170 effective words) took 0.0s, 751742 effective words/s
2019-06-19 16:24:53,042 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-06-19 16:24:53,045 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-06-19 16:24:53,046 : INFO : worker thread f

2019-06-19 16:24:53,406 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-06-19 16:24:53,408 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-06-19 16:24:53,409 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-06-19 16:24:53,410 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-06-19 16:24:53,410 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-06-19 16:24:53,411 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-06-19 16:24:53,413 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-06-19 16:24:53,417 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-06-19 16:24:53,420 : INFO : EPOCH - 10 : training on 23456 raw words (19208 effective words) took 0.0s, 1030815 effective words/s
2019-06-19 16:24:53,429 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-06-19 16:24:53,438 : INFO : worker thread

2019-06-19 16:24:53,821 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-06-19 16:24:53,824 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-06-19 16:24:53,827 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-06-19 16:24:53,828 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-06-19 16:24:53,830 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-06-19 16:24:53,831 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-06-19 16:24:53,834 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-06-19 16:24:53,838 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-06-19 16:24:53,843 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-06-19 16:24:53,845 : INFO : EPOCH - 18 : training on 23456 raw words (19131 effective words) took 0.0s, 531913 effective words/s
2019-06-19 16:24:53,857 : INFO : worker thread 

2019-06-19 16:24:54,168 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-06-19 16:24:54,171 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-06-19 16:24:54,173 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-06-19 16:24:54,174 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-06-19 16:24:54,177 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-06-19 16:24:54,179 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-06-19 16:24:54,181 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-06-19 16:24:54,182 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-06-19 16:24:54,185 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-06-19 16:24:54,189 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-06-19 16:24:54,192 : INFO : EPOCH - 26 : training on 23456 raw words (19209 effective

...

2019-06-19 16:24:56,421 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-06-19 16:24:56,422 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-06-19 16:24:56,425 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-06-19 16:24:56,428 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-06-19 16:24:56,429 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-06-19 16:24:56,430 : INFO : EPOCH - 73 : training on 23456 raw words (19108 effective words) took 0.0s, 874403 effective words/s
2019-06-19 16:24:56,449 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-06-19 16:24:56,452 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-06-19 16:24:56,454 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-06-19 16:24:56,456 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-06-19 16:24:56,457 : INFO : worker thread 

2019-06-19 16:24:56,791 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-06-19 16:24:56,794 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-06-19 16:24:56,796 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-06-19 16:24:56,800 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-06-19 16:24:56,802 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-06-19 16:24:56,804 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-06-19 16:24:56,808 : INFO : EPOCH - 81 : training on 23456 raw words (19135 effective words) took 0.0s, 774723 effective words/s
2019-06-19 16:24:56,822 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-06-19 16:24:56,828 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-06-19 16:24:56,831 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-06-19 16:24:56,834 : INFO : worker thread 

2019-06-19 16:24:57,152 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-06-19 16:24:57,154 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-06-19 16:24:57,157 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-06-19 16:24:57,159 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-06-19 16:24:57,160 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-06-19 16:24:57,163 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-06-19 16:24:57,176 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-06-19 16:24:57,178 : INFO : EPOCH - 89 : training on 23456 raw words (19158 effective words) took 0.0s, 547644 effective words/s
2019-06-19 16:24:57,204 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-06-19 16:24:57,207 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-06-19 16:24:57,209 : INFO : worker thread 

2019-06-19 16:24:57,541 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-06-19 16:24:57,542 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-06-19 16:24:57,544 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-06-19 16:24:57,546 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-06-19 16:24:57,548 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-06-19 16:24:57,551 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-06-19 16:24:57,553 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-06-19 16:24:57,554 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-06-19 16:24:57,556 : INFO : EPOCH - 97 : training on 23456 raw words (19159 effective words) took 0.0s, 602838 effective words/s
2019-06-19 16:24:57,569 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-06-19 16:24:57,577 : INFO : worker thread 

(1918475, 2345600)

In [148]:
w1 = "ABCG2".lower()

print("Most similar to {0}".format('ABCG2'), model.wv.most_similar(positive=w1))

Most similar to ABCG2 [('transporter', 0.8971574306488037), ('gefitinib', 0.8629952073097229), ('extrusion', 0.8603367209434509), ('causes', 0.8382961750030518), ('away', 0.8380654454231262), ('concomitant', 0.8371683359146118), ('membrane,', 0.8368896245956421), ('doxorubicin', 0.8220955729484558), ('inhibition', 0.8169850707054138), ('egfr', 0.8140828609466553)]


In [149]:
w1 = "glaucoma".lower()

print("Most similar to {0}".format('glaucoma'), model.wv.most_similar(positive=w1))

Most similar to glaucoma [('open-angle', 0.9977099299430847), ('(poag).', 0.9970671534538269), ('loxl1', 0.6293195486068726), ('population.', 0.5295864343643188), ('investigated.', 0.5186614990234375), ('thoroughly', 0.5030410289764404), ('recently,', 0.48044827580451965), ('been', 0.4706741273403168), ('middle', 0.4550781846046448), ('cyp1a1', 0.4542720317840576)]


In [150]:
w1 = "ias".lower()

print("Most similar to {0}".format(w1), model.wv.most_similar(positive=w1))

Most similar to ias [('rs42524', 0.9814943671226501), ('ethnicity.', 0.9536105394363403), ('col1a2', 0.8689796328544617), ('han', 0.8114165663719177), ('sporadic', 0.7153281569480896), ('chinese', 0.7105603218078613), ('authors', 0.6703982949256897), ('polymorphism', 0.5799390077590942), ('predicts', 0.5696506500244141), ('cyp3a5,', 0.563604474067688)]


In [143]:
word = 'glaucoma'
new_sentence = word.split()
model.wv.most_similar(positive=filter(lambda x: x in model.wv.vocab, new_sentence), topn=100)


[('open-angle', 0.941875159740448),
 ('(poag).', 0.9250666499137878),
 ('garcinia', 0.8112842440605164),
 ('primary', 0.8009686470031738),
 ('kola', 0.7902020812034607),
 ('stx2vha.', 0.7898602485656738),
 ('anti-hcv', 0.7787668108940125),
 ('intracranial', 0.7740756869316101),
 ('inactivate', 0.765733540058136),
 ('outcomes', 0.7632365822792053),
 ('instilled', 0.7628531455993652),
 ('aneurysms', 0.7626500725746155),
 ('old', 0.7582007646560669),
 ('2c9)', 0.758067786693573),
 ('occurred.', 0.7561087608337402),
 ('fluconazole.', 0.7476499080657959),
 ('1.04-1.52).', 0.746239185333252),
 ('cerebrovascular', 0.7415264248847961),
 ('disorder.', 0.7408965826034546),
 ('nut', 0.7371277809143066),
 ('pds.', 0.7356117963790894),
 ('hypertension.', 0.7261499166488647),
 ('exercise-induced', 0.7218173146247864),
 ('familial', 0.7216811180114746),
 ('(and/or', 0.7207029461860657),
 ('0.047).', 0.7203524708747864),
 ('cvcr', 0.7195112705230713),
 ('tests', 0.718197762966156),
 ('duplication.', 0